In [23]:
import NewCore
import os
os.chdir('/Volumes/FAIR/DC-Image-Analysis/DC-project-master/cleancode/NewMidplane')
import Functions
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from scipy.ndimage.interpolation import rotate
%matplotlib inline


from scipy.ndimage.morphology import binary_opening as bo
from scipy.ndimage.morphology import binary_erosion as be
from scipy.ndimage.morphology import binary_closing as bc
from scipy.ndimage.morphology import binary_fill_holes as bfh

import skimage
from skimage import filters

 
def BiggestRegion(array):
    labels,num = skimage.measure.label(array, connectivity=3,return_num='Ture')
    if num>1:
        props = skimage.measure.regionprops(labels)
        b = [p.area for p in props]
        ind = b.index(max(b))
        bcoords = props[ind].coords
        b_loop = np.zeros((array.shape))
        for i in range(len(bcoords)):
                a,b,c = bcoords[i]
                b_loop[a,b,c]=1
    else:
        b_loop = array
        
    return b_loop

from ipywidgets import interact, fixed
from IPython.display import display

def wl_to_lh(window, level):
    low = level - window/2
    high = level + window/2
    return low,high

import skimage
from skimage import filters

def display_image(img, x=None, y=None, z=None, window=None, level=None):
    # Convert SimpleITK image to NumPy array
    img_array = sitk.GetArrayFromImage(img)
    
    # Get image dimensions in millimetres
    size = img.GetSize()
    spacing = img.GetSpacing()
    width  = size[0] * spacing[0]
    height = size[1] * spacing[1]
    depth  = size[2] * spacing[2]
    
    if x is None:
        x = np.floor(size[0]/2).astype(int)
    if y is None:
        y = np.floor(size[1]/2).astype(int)
    if z is None:
        z = np.floor(size[2]/2).astype(int)
    
    if window is None:
        window = np.max(img_array) - np.min(img_array)
    
    if level is None:
        level = window / 2 + np.min(img_array)
    
    low,high = wl_to_lh(window,level)

    # Display the orthogonal slices
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

    ax1.imshow(img_array[z,:,:], cmap='gray', clim=(low, high), extent=(0, width, height, 0))
    ax2.imshow(img_array[:,y,:], origin='lower', cmap='gray', clim=(low, high), extent=(0, width,  0, depth))
    ax3.imshow(img_array[:,:,x], origin='lower', cmap='gray', clim=(low, high), extent=(0, height, 0, depth))

    # Additionally display crosshairs
    ax1.axhline(y * spacing[1], lw=1)
    ax1.axvline(x * spacing[0], lw=1)
    
    ax2.axhline(z * spacing[2], lw=1)
    ax2.axvline(x * spacing[0], lw=1)
    
    ax3.axhline(z * spacing[2], lw=1)
    ax3.axvline(y * spacing[1], lw=1)

    plt.show()
    
def reshape(array, ConstPixelSpacing):
    """ Reshape array to have cubic voxels of size 1mm^3 """
    '''mind the a, b, c value is adjusted to make each '''

    width = array.shape[0]
    height = array.shape[1]
    depth = array.shape[2]
    a = int((width)*ConstPixelSpacing[0])
    b = int((height)*ConstPixelSpacing[1])
    c = int((depth)*ConstPixelSpacing[2])
    reshapedArray1 = np.zeros((a,height,depth))
    xp = np.linspace(0, (width-1)*ConstPixelSpacing[0], width) #adjust xp as [0,1ConstPixelSpacing,2CPS..
    x  = np.linspace(0, a-1, a)

    for j in range(height):
        for k in range(depth):
            reshapedArray1[:,j,k] = np.interp(x, xp, array[:,j,k])
    reshapedArray2 = np.zeros((a,b,depth))
    yp = np.linspace(0,(height-1)*ConstPixelSpacing[1],height)
    y = np.linspace(0,b-1,b)
    for j in range(a):
        for k in range(depth):
            reshapedArray2[j,:,k] = np.interp(y, yp, reshapedArray1[j,:,k])
    reshapedArray3 = np.zeros((a,b,c))
    zp = np.linspace(0,(depth-1)*ConstPixelSpacing[2],depth)
    z = np.linspace(0,c-1,c)
    for j in range(a):
        for k in range(b):
            reshapedArray3[j,k,:] = np.interp(z, zp, reshapedArray2[j,k,:])
    return reshapedArray3
    


def saveImage(array,img):
    array = np.swapaxes(array,0,2)
    mask = sitk.GetImageFromArray(array)
    mask.CopyInformation(img)
    sitk.WriteImage(mask, os.path.split(path)[0] + '/ROI.nii.gz')



# paths = ['/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/SDH2/',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/SDH4/',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/SDH7/',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/SDH8/',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/SDH9/',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/SDH11/',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/SDH12/',
#         '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/SDH14/',
#         '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/SDH16-2/',
#         '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/SDH24/',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/CONT4/',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/CONT5/',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/CONT7/',
#         '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/EDH7/',
#         '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/Sample3_Pre/',
#         '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/Sample3_Post/',
#         '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/Sample4_Pre/',
#         '/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/tutorial3/']

# paths = ['/Volumes/FAIR/Eric_Midlinedeliniation/MPS1',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/MPS2',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/MPS3',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/MPS4',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/MPS5',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/MPS6',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/MPS7',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/MPS8',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/MPS9',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/MPS10',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/MPS11',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/MPS12',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/MPS13',
#          '/Volumes/FAIR/Eric_Midlinedeliniation/MPS14']

paths = ['/Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/tutorial3']

for i in range(len(paths)):
    path = paths[i]
    print( "Reading directory:", path)

    img = sitk.ReadImage(path + '/New_MPS.nii.gz')
    img_array = sitk.GetArrayFromImage(img)
    ConstPixelSpacing = img.GetSpacing()

    mps_array = np.multiply(img_array==2.,img_array)
    mps_img = sitk.GetImageFromArray(mps_array)
    mps_img.CopyInformation(img)
    sitk.WriteImage(mps_img, os.path.split(path)[0] + '/mps.nii.gz')

Reading directory: /Volumes/FAIR/Eric_Midlinedeliniation/Self-MPS/tutorial3


In [21]:
mps_array[mps_array>0]

array([ 2.,  2.,  2., ...,  2.,  2.,  2.])